In [15]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

In [16]:
# 1. Cargar los datasets
pathSmall = '../Datasets/ml-latest-small/'
path1M = '../Datasets/ml-1m/'
pathLarge = '../Datasets/ml-32m/'
ratings = pd.read_csv(pathSmall + 'ratings.csv')  # Debe tener columnas: userId, movieId, rating, timestamp
#ratings = pd.read_csv(path1M + 'ratings.dat', sep='::', header=None, engine='python')
ratings.columns = ['userId', 'movieId', 'rating', 'timestamp']  # Asignar nombres a las columnas
ratings.drop(columns=['timestamp'], inplace=True)  # Eliminar la columna timestamp

In [17]:
def predict_rating(user_index, item_index, data, clusters, user_means):
    cluster_id = clusters[user_index]
    cluster_members = np.where(clusters == cluster_id)[0]
    
    votes = []
    user_vector = data[user_index]
    user_mean = user_means[user_index]

    for neighbor in cluster_members:
        if neighbor == user_index:
            continue
        neighbor_rating = data[neighbor, item_index]
        if neighbor_rating == 0:
            continue

        neighbor_mean = user_means[neighbor]
        deviation = neighbor_rating - neighbor_mean

        distance = np.linalg.norm(user_vector - data[neighbor])
        weight = 1 / (distance**2 + 1e-5)

        votes.append((deviation, weight))
    
    if not votes:
        return user_mean  # Si no hay votos, devolver la media del usuario

    # Promedio ponderado de las desviaciones
    num = sum(dev * w for dev, w in votes)
    denom = sum(w for _, w in votes)

    pred = user_mean + (num / denom)
    return np.clip(pred, 0.5, 5.0)  # Asegura que el rating esté en el rango válido

In [18]:
# 2. Crear matriz usuario-película
# Pivot table: filas = usuarios, columnas = películas, valores = rating
ratings_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

In [19]:
# Parámetros para KMeans y 5-fold cross validation
k = 5
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = []

# Convertir el DataFrame en un array de índices para iterar en el splitting
ratings_indices = ratings.index.values

In [20]:
# 2. Iterar sobre cada fold
for train_indices, test_indices in kf.split(ratings_indices):
    
    # Crear conjuntos de entrenamiento y prueba a partir de los índices
    train_ratings = ratings.iloc[train_indices]
    test_ratings = ratings.iloc[test_indices]
    
    # Construir la matriz usuario-película a partir del conjunto de entrenamiento
    train_matrix = train_ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

    # Calcular la media por usuario para ajustar como KNNWithMeans
    user_means = train_matrix.replace(0, np.NaN).mean(axis=1).values
    
    # Realizar clustering en la matriz de entrenamiento
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters_train = kmeans.fit_predict(train_matrix)
    
    # Listas para acumular predicciones y valores reales
    predictions = []
    true_values = []
    
    # Convertir índices de usuarios y películas para facilitar búsqueda
    usuarios = list(train_matrix.index)
    peliculas = list(train_matrix.columns)
    train_data = train_matrix.values
    
    # Iterar sobre cada registro del conjunto de prueba
    for idx, row in test_ratings.iterrows():
        user = row['userId']
        movie = row['movieId']
        true_rating = row['rating']
        
        # Si el usuario o la película no están en el conjunto de entrenamiento, se omite el registro
        if user not in usuarios or movie not in peliculas:
            continue
        
        user_idx = usuarios.index(user)
        movie_idx = peliculas.index(movie)
        
        pred = predict_rating(user_idx, movie_idx, train_data, clusters_train, user_means)
        predictions.append(pred)
        true_values.append(true_rating)
    
    # Calcular el MAE para el fold actual, si existen predicciones
    if predictions:
        mae_fold = np.mean(np.abs(np.array(predictions) - np.array(true_values)))
        mae_scores.append(mae_fold)
        print(f"Fold MAE: {mae_fold:.4f}")

# Mostrar el MAE y NMAE promedio en todos los folds
if mae_scores:
    mae_promedio = np.mean(mae_scores)
    nmae_promedio = mae_promedio / 4.5  # Rango de ratings en MovieLens
    print(f"\nMAE promedio en 5-fold cross validation: {mae_promedio:.4f}")
    print(f"NMAE promedio en 5-fold cross validation: {nmae_promedio:.4f}")
else:
    print("No se realizaron predicciones en ninguno de los folds.")

Fold MAE: 0.7109
Fold MAE: 0.6987
Fold MAE: 0.7068
Fold MAE: 0.7104
Fold MAE: 0.7056

MAE promedio en 5-fold cross validation: 0.7065
NMAE promedio en 5-fold cross validation: 0.1570
